# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **FocalLoss** for class-imbalanced learning

**Target Metrics:**
- Specificity: 98.72%
- G-Means Improvement: 18.11%
- P95 Latency: <100ms

## 1️⃣ Setup Environment

In [1]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

Cloning into 'fraudguard'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 154 (delta 60), reused 125 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (154/154), 104.58 KiB | 5.23 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/fraudguard


In [3]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

import torch

# 1. Get exact versions
pt_version = torch.__version__.split('+')[0]  # e.g., 2.5.1
cuda_version = "cu" + torch.version.cuda.replace('.', '')  # e.g., cu124
wheel_url = f"https://data.pyg.org/whl/torch-{pt_version}+{cuda_version}.html"

print(f"PyTorch: {pt_version}, CUDA: {cuda_version}")
print(f"Downloading from: {wheel_url}")

# 2. Install with visible output (force reinstall to fix broken partial installs)
!pip install --force-reinstall torch-scatter torch-sparse -f $wheel_url

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00
⚠️ faiss-gpu not available, using faiss-cpu
   (Graph building on CPU, but GNN training still runs on GPU!)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 104.6 MB/s eta 0:00:00
PyTorch: 2.9.0, CUDA: cu126
Looking in links: https://data.pyg.org/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 133.1 MB/s eta 0:00:00


Obtaining file:///content/fraudguard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.4 MB/s eta 0:00:00
  Building editable for fraudguard (pyproject.toml) ... done
  Created wheel for fraudguard: filename=fraudguard-0.1.0-py3-none-any.whl size=2801 sha256=5638c0b714acd4b297203f70f102910966a1da4432de3a3f589a22ae37244e19
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ob56ewf/wheels/c6/29/62/fb6d8d095576e7e3efddf4fdcb7dfc799af71ace273f1ee84c
Successfully built fraudguard
  Attempting uninstall: structlog
    Found existing installation: structlog 25.5.0
    Uninstalling structlog-25.5.0:
      Successfully unins


✓ Environment setup complete


In [1]:
import torch
try:
    import torch_scatter
    import torch_sparse
    import fraudguard
    print("✅ Success! Libraries are installed and loaded.")
except ImportError as e:
    print(f"❌ Still missing libraries: {e}")
    # Only if you see this error should you go back and install again.

✅ Success! Libraries are installed and loaded.


## 2️⃣ Configuration

In [2]:
import os

# ==============================================
# CONFIGURATION - UPDATE THESE PATHS AS NEEDED
# ==============================================

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 1.0      # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 30
BATCH_SIZE = 4096      # Reduce to 2048 or 1024 if OOM
NUM_NEIGHBORS = [25, 10]  # 2-hop neighborhood sampling

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

Data: /content/drive/MyDrive/ieee-fraud-detection
Models: /content/drive/MyDrive/fraudguard-models
Logs: /content/drive/MyDrive/fraudguard-logs

Batch size: 4096
Sample fraction: 100%


## 3️⃣ Verify GPU and FAISS

In [3]:
import torch
import faiss

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
VRAM: 15.8 GB

✓ GNN training will run on GPU

FAISS GPUs: 0
   (Using CPU FAISS for graph building - this is OK)


## 4️⃣ Load and Preprocess Data

In [4]:
import sys
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from src.data.loader import FraudDataLoader
from src.data.preprocessor import FeaturePreprocessor
from src.data.graph_builder import GraphBuilder
from src.utils.config import load_data_config
from src.utils.device_utils import set_seed, get_device

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

Loading faiss with CPU support (no GPU detected).
Using device: cuda

Data loaded:
  Train: 354,324
  Val: 118,108
  Test: 118,108
  Fraud rate: 3.50%


## 5️⃣ Run Full AD-RL-GNN Pipeline

We use the `FraudTrainer` class to orchestrate the full pipeline, including:
1. **AdaptiveMCD**: Intelligent majority downsampling
2. **RL Agent**: Dynamic subgraph selection (Random Walk, K-Hop, K-Ego)
3. **Graph Enhancement**: Adding semantic edges
4. **GNN Training**: CrossEntropyLoss (15x weight)

In [14]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from src.training.trainer import FraudTrainer
from src.utils.config import load_model_config, load_data_config
from sklearn.metrics import confusion_matrix
import numpy as np
import time

model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# A. DATASET SIZE (20% Subset)
SAMPLE_FRAC_TEST = 0.20

# B. TUNED PARAMETERS (ABLATION STUDY)
model_cfg.training["max_epochs"] = 30
model_cfg.training["learning_rate"] = 0.005  # Slightly higher LR for convergence
model_cfg.adaptive_mcd["alpha"] = 0.75       # Moderate Drop (~60-70% majority removed)
model_cfg.graph.similarity_threshold = 0.80  # Strict graph (Clean edges)

# Initialize Trainer
print(f"🚀 Initializing ABLATION Run (No RL) on {SAMPLE_FRAC_TEST*100}% Data...")
trainer = FraudTrainer(
    model_config=model_cfg,
    data_config=data_cfg,
    device=device
)

print("📉 Loading 20% Data Subset...")
loader = FraudDataLoader(config=data_cfg)
df_subset = loader.load_train_data(sample_frac=SAMPLE_FRAC_TEST)
train_df_sub, val_df_sub, test_df_sub = loader.create_splits(df_subset)

print("⚙️ Building Graph (0.80)...")
trainer._preprocess(train_df_sub, val_df_sub, test_df_sub)
trainer._build_graph()
trainer._prepare_labels(train_df_sub, val_df_sub, test_df_sub)

print("\n🧠 Training AdaptiveMCD (Alpha=0.75)...")
trainer._train_mcd()

# ❌ DISABLED FOR THIS TEST (ABLATION)
# print("\n🤖 Training RL Agent...")
# trainer._train_rl_and_enhance()
print("\n🚫 SKIPPING RL Agent (Testing Baseline Stability)...")

# CHANGED: Weight 22.0 (High bias to overcome 8:1 imbalance)
print("\n⚖️ Applying Weight 22.0...")
weights = torch.tensor([1.0, 22.0]).to(device)

trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
criterion = trainer.criterion
model = trainer.model

# Optimizer
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.005,
    weight_decay=1e-4
)

print("\n📦 Setting up Loaders...")
optimized_data = Data(
    x=trainer.X_full,
    edge_index=trainer.edge_index,
    y=trainer.all_labels
)
optimized_data.train_mask = trainer.train_mask
optimized_data.val_mask = trainer.val_mask
optimized_data.test_mask = trainer.test_mask

train_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=2048,
    input_nodes=optimized_data.train_mask,
    shuffle=True
)

val_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=2048,
    input_nodes=optimized_data.val_mask,
    shuffle=False
)

print(f"\n🚀 Starting Training (Goal: Stable convergence)...")
best_gmeans = 0

for epoch in range(30):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

        # Keep Gradient Clipping
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    if epoch % 1 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)

        print(f"Epoch {epoch+1:>3} | Loss: {avg_loss:.4f} | Spec: {tnr*100:.2f}% | Recall: {tpr*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans:
            best_gmeans = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_subset_test.pt")

print(f"\nTest Complete. Best G-Means: {best_gmeans*100:.2f}%")

🚀 Initializing ABLATION Run (No RL) on 20.0% Data...
📉 Loading 20% Data Subset...
⚙️ Building Graph (0.80)...

🧠 Training AdaptiveMCD (Alpha=0.75)...

🚫 SKIPPING RL Agent (Testing Baseline Stability)...

⚖️ Applying Weight 22.0...

📦 Setting up Loaders...

🚀 Starting Training (Goal: Stable convergence)...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(


Epoch   1 | Loss: 0.7873 | Spec: 47.74% | Recall: 51.38% | G-Means: 49.53%
Epoch   2 | Loss: 0.6874 | Spec: 50.42% | Recall: 51.59% | G-Means: 51.00%
Epoch   3 | Loss: 0.6724 | Spec: 32.35% | Recall: 67.52% | G-Means: 46.73%
Epoch   4 | Loss: 0.6692 | Spec: 52.78% | Recall: 47.35% | G-Means: 49.99%
Epoch   5 | Loss: 0.6675 | Spec: 52.81% | Recall: 47.66% | G-Means: 50.17%
Epoch   6 | Loss: 0.6640 | Spec: 46.19% | Recall: 53.29% | G-Means: 49.61%
Epoch   7 | Loss: 0.6612 | Spec: 57.53% | Recall: 41.72% | G-Means: 48.99%
Epoch   8 | Loss: 0.6563 | Spec: 46.85% | Recall: 53.08% | G-Means: 49.87%
Epoch   9 | Loss: 0.6532 | Spec: 49.46% | Recall: 50.42% | G-Means: 49.94%
Epoch  10 | Loss: 0.6534 | Spec: 46.64% | Recall: 51.70% | G-Means: 49.11%
Epoch  11 | Loss: 0.6487 | Spec: 51.49% | Recall: 47.77% | G-Means: 49.60%
Epoch  12 | Loss: 0.6477 | Spec: 39.82% | Recall: 60.72% | G-Means: 49.17%
Epoch  13 | Loss: 0.6467 | Spec: 55.06% | Recall: 45.65% | G-Means: 50.13%
Epoch  14 | Loss: 0.6425 

## 6️⃣ Evaluation & Claims Verification

In [ ]:
# Safe Evaluation (Mini-Batch)
print("🔍 Benchmarking & Evaluation...")
model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_best_pro.pt"))
model.eval()

# Use neighbors=[25, 10] for fast inference (Latency < 10ms)
test_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=4096,
    input_nodes=optimized_data.test_mask,
    shuffle=False
)

latencies = []
all_preds, all_true = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        out = model(batch.x, batch.edge_index)
        latencies.append((time.perf_counter() - start) * 1000)
        pred = out[:batch.batch_size].argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

# Metrics
cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
gmeans = np.sqrt(tpr * tnr)
p95_latency = np.percentile(latencies, 95)

print("="*60)
print("FINAL RESULTS (Pro Framework)")
print("="*60)
print(f"Specificity: {tnr*100:.2f}%")
print(f"Recall:      {tpr*100:.2f}%")
print(f"G-Means:     {gmeans*100:.2f}%")
print(f"P95 Latency: {p95_latency:.2f} ms")

🔍 Benchmarking & Evaluation...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(


FINAL RESULTS (Pro Framework)
Specificity: 63.24%
Recall:      34.47%
G-Means:     46.69%
P95 Latency: 45.51 ms


In [15]:
import time
from torch_geometric.loader import NeighborLoader

print("🔍 Benchmarking Stability on 20% Test Set...")

# 1. Load the model trained on the subset
# Note: We load 'fraudguard_subset_test.pt', not 'best_model.pt'
model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_subset_test.pt"))
model.eval()

# 2. Create a Test Loader linked to the OPTIMIZED data
# Note: We use 'optimized_data', not 'data'
test_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],   # Standard inference neighbors
    batch_size=4096,
    input_nodes=optimized_data.test_mask,
    shuffle=False
)

# 3. Run Inference
latencies = []
all_preds, all_true = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        out = model(batch.x, batch.edge_index)
        latencies.append((time.perf_counter() - start) * 1000)

        pred = out[:batch.batch_size].argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

# 4. Compute Metrics
cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
gmeans = np.sqrt(tpr * tnr)
p95_latency = np.percentile(latencies, 95)

print("="*60)
print("STABILITY TEST RESULTS (20% Data)")
print("="*60)
print(f"Confusion Matrix: TP={tp}, TN={tn}, FP={fp}, FN={fn}")
print(f"Specificity: {tnr*100:.2f}%")
print(f"Recall:      {tpr*100:.2f}%")
print(f"G-Means:     {gmeans*100:.2f}%")
print(f"P95 Latency: {p95_latency:.2f} ms")

🔍 Benchmarking Stability on 20% Test Set...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(


STABILITY TEST RESULTS (20% Data)
Confusion Matrix: TP=412, TN=11343, FP=11489, FN=378
Specificity: 49.68%
Recall:      52.15%
G-Means:     50.90%
P95 Latency: 21.55 ms


## 7️⃣ Save Model

In [ ]:
trainer.save(f"{MODELS_DIR}/fraudguard_full_pipeline.pt")
print(f"Model saved to {MODELS_DIR}/fraudguard_full_pipeline.pt")

Model saved to /content/drive/MyDrive/fraudguard-models/fraudguard_full_pipeline.pt


In [ ]:
from google.colab import runtime
runtime.unassign()